# 07 プランニング: Semantic Kernel 連携サンプル (日本語版)

英語版 `07-planning-design/code_samples/07-semantic-kernel.ipynb` を基に、Semantic Kernel を使って
「ゴール文 → サブタスク分解(JSON) → 検証」という最小パイプラインを段階的に学べる形へ再構成します。

### 目的
ユーザーが自然言語でエージェントに実施させたいことをゴールとして入力し、先に定義した Pydantic スキーマ (Plan/SubTask/AgentEnum) に準拠した構造化 JSON を
LLM から取得・検証し、後続フェーズ (ルーティング / 実行 / Re-plan) への土台を形成する。

### 得られる成果物
- 先行スキーマ定義による “出力スキーマ” の固定と妥当性検証パターン
- Semantic Kernel における ChatCompletion サービス登録と関数化プロンプト
- JSON 生成 ～ パース ～ Pydantic 検証 ～ サブタスク列挙の最小パイプライン
- README 「タスク分解」「構造化出力」セクションとの対応理解

### セル構成 (英語版を日本語学習向けに拡張)
| セル | 種別 | 目的 | 主な要素 |
|------|------|------|----------|
| 1 | Markdown | 学習ゴール/構成提示 | 目的 / 成果物 / 対応表 |
| 2 | Code | モジュール インストール & インポート & 出力スキーマ固定 | %pip / Enum / Pydantic |
| 3 | Code | Kernel + サービス初期化 | AzureChatCompletion 登録 |
| 4 | Code | プロンプト関数化 | create_function_from_prompt |
| 5 | Code | 推論・JSON抽出・検証 | invoke_async / JSON / Validation |
| 6 | Markdown | 次ステップ | ルーティング / Re-plan / 品質評価 |

※ 英語版は 1 Markdown + 1 大きなコードセル + 終端 Markdown だったが、学習段階を明示するため細分化。

### README との対応
- タスク分解: セル2 スキーマ定義 + セル5 で subtasks の生成/表示
- 構造化出力: セル5 の JSON 抽出 + Pydantic 検証
- 再計画 (Re-plan): 未実装 (Plan と履歴差分を扱うセル追加で拡張可能)
- マルチエージェント: subtasks.assigned_agent を後続ルーティング層へ受け渡しできる形に保持

### 実行手順
1. セル2: 依存パッケージとスキーマ定義
2. セル3: Kernel と ChatCompletion サービス追加
3. セル4: プラン生成プロンプトを関数化 (PROMPT 内の JSON 指示を調整可)
4. セル5: goal を設定して invoke_async → JSON 抽出 → 検証 → サブタスク列挙
5. System Prompt や goal を変更し再実行し差異を観察
6. セル6 の拡張案を参照してルーティング/再計画を追加

---


In [5]:
%pip install "semantic-kernel>=1.35.0,<2.0" pydantic azure-ai-inference --quiet
# ↑ 旧 dev 版(0.9.0.dev0)は Python3.13 非対応。1.35.x 系は 3.13 で利用可。

# =============================================
# セル2: 依存インストール & インポート & 出力スキーマ
# 目的: 先に“出力スキーマ”を固定し LLM の自由度を制限 -> 後続処理(ルーティング/再実行)を安定化
# 成果物(1/5): スキーマ定義完了 -> ARTIFACT_SCHEMA_DEFINED=True で可視化
# =============================================
import os, json, re
from pydantic import BaseModel, Field, ValidationError  # BaseModel 継承クラスの model_validate()/model_validate_json() で受信 JSON を厳格に検証
from typing import List
from enum import Enum

# --- エージェント種別列挙 (後段ルーティング前提) ---
class AgentEnum(str, Enum):
    Flight='flight_booking'
    Hotel='hotel_booking'
    Car='car_rental'
    Activity='activities_booking'
    Destination='destination_info'
    # default_agent 相当は未使用。必要なら追加。

# --- サブタスク要素 ---
class SubTask(BaseModel):
    task_details: str = Field(min_length=3, description='後工程がそのまま実行可能な粒度')
    assigned_agent: AgentEnum

# --- プラン全体 ---
class Plan(BaseModel):
    main_task: str
    subtasks: List[SubTask]
    is_greeting: bool

ARTIFACT_SCHEMA_DEFINED = True  # 成果物1 達成フラグ
print('[SCHEMA] Plan/SubTask 出力スキーマ定義完了 -> ARTIFACT_SCHEMA_DEFINED=True')


Note: you may need to restart the kernel to use updated packages.
[SCHEMA] Plan/SubTask 出力スキーマ定義完了 -> ARTIFACT_SCHEMA_DEFINED=True



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
# =============================================
# セル3: Kernel + ChatCompletion サービス初期化
# 目的: Semantic Kernel に AzureChatCompletion サービスを登録し推論呼び出しの基盤を作る
# 成果物(2/5): サービス初期化 -> ARTIFACT_SERVICE_INITIALIZED=True
# =============================================
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# トークンは GitHub Models (Azure AI Inference) を想定
TOKEN = os.getenv('GITHUB_TOKEN') or os.getenv('OPENAI_API_KEY')
if not TOKEN:
    raise RuntimeError('GITHUB_TOKEN / OPENAI_API_KEY が必要です (.env を確認)')
ENDPOINT = os.getenv('OPENAI_BASE_URL','https://models.inference.ai.azure.com')
MODEL = os.getenv('OPENAI_MODEL','gpt-4o-mini')

kernel = sk.Kernel()
# インスタンスを保持して後続セルで直接参照 (API 変更で列挙取得が不安定なため)
chat_service = AzureChatCompletion(deployment_name=MODEL, endpoint=ENDPOINT, api_key=TOKEN)
kernel.add_service(chat_service)
# グローバル参照用 (セル4で利用)
CHAT_SERVICE = chat_service
ARTIFACT_SERVICE_INITIALIZED = True
print(f'[KERNEL] サービス登録完了 model={MODEL} -> ARTIFACT_SERVICE_INITIALIZED=True (CHAT_SERVICE ready)')


[KERNEL] サービス登録完了 model=gpt-4o-mini -> ARTIFACT_SERVICE_INITIALIZED=True (CHAT_SERVICE ready)


In [18]:
# =============================================
# セル4: プロンプト関数 (generate_plan) 定義 (ChatHistory 対応版)
# 目的: ChatHistory を用い get_chat_message_contents(chat_history, settings) を正しく呼び出し JSON 計画文字列を取得
# 修正: 直列 messages -> ChatHistory へ移行 (前エラーは chat_history 引数不足)
# =============================================
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory
import asyncio

try:
    _chat_service = CHAT_SERVICE  # type: ignore
except NameError:
    raise RuntimeError('CHAT_SERVICE が未定義です。セル3を先に実行してください。')
if not isinstance(_chat_service, AzureChatCompletion):
    raise RuntimeError('CHAT_SERVICE が AzureChatCompletion インスタンスではありません')

PROMPT_TEMPLATE = (
    "あなたは旅行計画を行うプランナーエージェントです。以下の要件でタスク分解した JSON を返してください。\n\n"
    "【絶対条件】\n"
    "- 出力は JSON オブジェクトのみ (説明や余計な文字列を含めない)\n"
    "- ルートキー: main_task, subtasks, is_greeting の3つのみ\n"
    "- main_task: ゴール全体を一文で要約\n"
    "- subtasks: 配列。各要素は {task_details, assigned_agent}。task_details は具体的手順。\n"
    "  * assigned_agent 値: flight_booking | hotel_booking | car_rental | activities_booking | destination_info\n"
    "- is_greeting: ユーザー入力が挨拶のみなら true, それ以外は false\n"
    "- 余計なキーを追加しない\n\n"
    "【最小サンプル】\n"
    "{\"main_task\":\"家族旅行計画\",\"subtasks\":[{\"task_details\":\"往復航空券候補の取得\",\"assigned_agent\":\"flight_booking\"}],\"is_greeting\":false}\n\n"
    "上記条件を満たす JSON を1つだけ返す。ゴール文: {{$input}}"
)

async def generate_plan(goal: str) -> str:
    system_content = PROMPT_TEMPLATE.replace('{{$input}}', goal)
    history = ChatHistory()
    # system メッセージ 1 本で指示 + ユーザー入力統合 (gpt-4o 系はこれで十分)
    history.add_system_message(system_content)

    settings = OpenAIChatPromptExecutionSettings(
        temperature=0,
        top_p=1,
        max_tokens=800,
    )
    try:
        results = await _chat_service.get_chat_message_contents(history, settings)
    except TypeError:
        # 署名が keyword 必須の場合
        results = await _chat_service.get_chat_message_contents(chat_history=history, settings=settings)  # type: ignore
    if not results:
        raise RuntimeError('空レスポンス')
    primary = getattr(results[0], 'content', None) or str(results[0])
    return primary or ''

ARTIFACT_PROMPT_FUNCTION_CREATED = True
print('[PROMPT] generate_plan 定義 (ChatHistory 対応) -> ARTIFACT_PROMPT_FUNCTION_CREATED=True')


[PROMPT] generate_plan 定義 (ChatHistory 対応) -> ARTIFACT_PROMPT_FUNCTION_CREATED=True


In [ ]:
# =============================================
# セル5: 推論 -> JSON抽出 -> 検証 -> 列挙 (generate_plan 利用 / ループ単純化)
# 目的: generate_plan(goal) で得た生テキストを JSON 化し Plan に検証 / 再試行
# 改訂: ChatHistory 版 generate_plan 対応。再試行時は goal に '#retry' を付与し system メッセージ再生成。
# =============================================
import asyncio, json, re
from collections import Counter

goal = '家族4人(子供2人) シンガポール→メルボルン 3日旅行プランを作成'
print('[STEP] Goal 受領 -> generate_plan 実行準備 (await パス)')

MAX_RETRIES = 3

async def obtain_plan(goal_text: str):
    mutable_goal = goal_text
    for attempt in range(1, MAX_RETRIES+1):
        try:
            raw_text = await generate_plan(mutable_goal)
            print(f'[RAW][TRY {attempt}]', (raw_text or '')[:200].replace('\n',' '), '...')
            if not raw_text:
                raise RuntimeError('空レスポンス')
            m = re.search(r'\{.*\}$', raw_text.strip(), re.DOTALL)
            json_str = m.group(0) if m else raw_text
            data = json.loads(json_str)
            plan_local = Plan.model_validate(data)
            print(f'[VALIDATION][OK] attempt={attempt} subtasks={len(plan_local.subtasks)}')
            return plan_local
        except (json.JSONDecodeError, ValidationError, RuntimeError) as e:
            print(f'[VALIDATION][RETRY] attempt={attempt} エラー: {e}')
            if attempt == MAX_RETRIES:
                raise
            mutable_goal += ' #retry'
    raise RuntimeError('再試行上限')

try:
    running_loop = asyncio.get_running_loop()
    if running_loop.is_running():
        plan = await obtain_plan(goal)  # type: ignore  # noqa
    else:
        plan = running_loop.run_until_complete(obtain_plan(goal))
except RuntimeError:
    plan = asyncio.run(obtain_plan(goal))

agent_counts = Counter(st.assigned_agent.value for st in plan.subtasks)
agent_summary = ', '.join(f"{k}:{v}" for k, v in agent_counts.items())
validation_ok = True
ARTIFACT_JSON_VALIDATED = validation_ok and len(plan.subtasks) >= 1
ARTIFACT_README_MAPPED = ('subtasks' in plan.model_dump()) and validation_ok

rows = [
    ('1/5 スキーマ固定', ARTIFACT_SCHEMA_DEFINED, 'Plan/SubTask Pydantic 定義'),
    ('2/5 サービス初期化', ARTIFACT_SERVICE_INITIALIZED, 'Kernel + AzureChatCompletion 登録'),
    ('3/5 関数定義', ARTIFACT_PROMPT_FUNCTION_CREATED, 'generate_plan 実装'),
    ('4/5 JSON検証', ARTIFACT_JSON_VALIDATED, '再試行 + model_validate 成功'),
    ('5/5 README対応', ARTIFACT_README_MAPPED, 'タスク分解 + 構造化出力')
]
print('\n[ARTIFACTS] 達成状況')
for title, flag, reason in rows:
    status = '✅' if flag else '❌'
    print(f'  {status} {title:<14} : {reason}')

print('\n[PLAN] main_task=', plan.main_task)
for i, st in enumerate(plan.subtasks, 1):
    print(f'  - #{i:02d} [{st.assigned_agent.value}] {st.task_details}')
print(f'\n[STATS] subtasks={len(plan.subtasks)} agents=({agent_summary}) is_greeting={plan.is_greeting}')


[STEP] Goal 受領 -> generate_plan 実行準備 (await パス)
[RAW][TRY 1] {"main_task":"家族4人(子供2人)のシンガポールからメルボルンへの3日旅行プランを作成","subtasks":[{"task_details":"往復航空券候補の取得","assigned_agent":"flight_booking"},{"task_details":"メルボルンの宿泊先の候補を探す","assigned_agent":"hotel_booking"},{"ta ...
[VALIDATION][OK] attempt=1 subtasks=5

[ARTIFACTS] 達成状況
  ✅ 1/5 スキーマ固定     : Plan/SubTask Pydantic 定義
  ✅ 2/5 サービス初期化    : Kernel + AzureChatCompletion 登録
  ✅ 3/5 関数定義       : generate_plan 実装
  ✅ 4/5 JSON検証     : 再試行 + model_validate 成功
  ✅ 5/5 README対応   : タスク分解 + 構造化出力

[PLAN] main_task= 家族4人(子供2人)のシンガポールからメルボルンへの3日旅行プランを作成
  - #01 [flight_booking] 往復航空券候補の取得
  - #02 [hotel_booking] メルボルンの宿泊先の候補を探す
  - #03 [car_rental] メルボルンでのレンタカーの手配
  - #04 [activities_booking] メルボルンでのアクティビティの予約
  - #05 [destination_info] メルボルンの観光情報を収集

[STATS] subtasks=5 agents=(flight_booking:1, hotel_booking:1, car_rental:1, activities_booking:1, destination_info:1) is_greeting=False

[TIPS] さらなる堅牢化:
- JSON Schema 指定 (structured_output) を組み込みレスポンス揺れを

### AutoGen ノートとの比較視点
| 視点 | Semantic Kernel | AutoGen |
|------|-----------------|---------|
| 最小コード行数 | 若干長い (関数化手順) | 短い (直接 create) |
| 再利用性 | create_function_from_prompt で高い | コード内に直書き (関数化は任意) |
| サービス切替 | kernel に複数サービス登録可能 | クライアント差し替え要 |
| プロンプト差し替え | 関数再生成で容易 | 文字列再代入 |
| 拡張ポイント | Middleware / Planner 拡張 | 会話エージェント連携 |